## Imports

In [41]:
import pandas as pd

# import spacy
import numpy as np
import sklearn as skl 
import xgboost as xgb


## Training

In [ ]:
target = 'label'
input_column = 'cleanTitle'

train_data = pd.read_pickle('../../../Files/Submissions/train/sets/train_split_submission.pickle') 
valid_data = pd.read_pickle('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_pickle('../../../Files/Submissions/train/sets/test_split_submission.pickle')


In [59]:
target = 'label'
input_column = 'cleanText'

train_data = pd.read_csv('../../../Files/Submissions/train/train2.csv', sep='|') 
# valid_data = pd.read_pi('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_csv('../../../Files/Submissions/train/test2.csv', sep='|')


In [53]:
train_data.shape

(1471, 14)

In [61]:
train_data.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments,cleanText
label,,,,,,,,,,,,,
0,426,426,426,426,426,426,164,426,426,426,426,0,426
1,619,619,619,619,619,619,121,619,619,619,619,2,619
2,426,426,426,426,426,426,121,426,426,426,426,0,426


In [54]:
target = 'label'
input_column = 'cleanText'

train_data = pd.read_csv('../../../Files/Submissions/train/train1.csv', sep='|') 
# valid_data = pd.read_pi('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_csv('../../../Files/Submissions/train/test1.csv', sep='|')

In [60]:
test_data.shape

(368, 14)

In [43]:

train_data = train_data[[target, input_column]]
# valid_data = valid_data[[target, input_column]]
test_data = test_data[[target, input_column]]

data = pd.concat([train_data, test_data])


train_instances = train_data[input_column].apply(str).apply(str.split)
train_labels = train_data[target]

# collect known word tokens and tags
wordset, labelset = set(), set()

# collect tags from all data, to prevent unseen labels
labelset.update(set(data[target]))

# get the vocabulary
for words in train_instances:
    wordset.update(set(words))

# map words and tags into ints
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
label2int = {label: i for i, label in enumerate(sorted(labelset))}
# inverted index to translate it back
int2label = {i:label for label, i in label2int.items()}


def convert2ints(instances):
    """
    function to apply the mapping to all words
    """
    result = []
    for words in instances:
        # replace words with int, 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        result.append(word_ints)
    return result
                          
train_instances_int = convert2ints(train_instances)
train_labels_int = [label2int[label] for label in train_labels]

In [44]:
test_instances = test_data[input_column].apply(str).apply(str.split)
test_labels = test_data[target]

test_instances_int = convert2ints(test_instances)
test_labels_int = [label2int[label] for label in test_labels]

# convert dev data
# val_instances = valid_data[input_column].apply(str).apply(str.split)
# val_labels = valid_data[target]

# val_instances_int = convert2ints(val_instances)
# val_labels_int = [label2int[label] for label in val_labels]

In [45]:
from keras.utils import to_categorical

train_labels_1hot = to_categorical(train_labels_int, len(label2int))
test_labels_1hot = to_categorical(test_labels_int, len(label2int))
# val_labels_1hot = to_categorical(val_labels_int, len(label2int))

train_labels_1hot[0]

array([0., 0., 1.], dtype=float32)

In [46]:
# compute 95th percentile of training sentence lengths
L = sorted(map(len, train_instances))
MAX_LENGTH = L[int(len(L)*0.95)]
print(MAX_LENGTH)

# apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_instances_int = pad_sequences(train_instances_int, padding='post', maxlen=MAX_LENGTH)
test_instances_int = pad_sequences(test_instances_int, padding='post', maxlen=MAX_LENGTH)
# val_instances_int = pad_sequences(val_instances_int, padding='post', maxlen=MAX_LENGTH)

print(train_instances[0], len(train_instances[0]))
print(train_instances_int[0], len(train_instances_int[0]))

88
['kansas', 'kdhe', 'report', 'new', 'covid', 'cluster', 'thanksgive', 'variant', 'surge', 'kansas', 'say', 'unvaccinated', 'staff', 'member', 'blame', 'uptick', 'outbreak', 'nursing', 'home'] 19
[2834 2842 4483 3481 1141  910 5286 5704 5157 2834 4665 5608 4997 3204
  567 5627 3731 3608 2417    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0] 88


In [47]:
print(len(test_instances_int)) 

368


In [9]:
dtrain = xgb.DMatrix(train_instances_int, label=train_labels_int)
# dval = xgb.DMatrix(val_instances_int, label=val_labels_int)
dtest = xgb.DMatrix(test_instances_int, label=test_labels_int)

In [20]:
param = {'max_depth': 8, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 2, 'eval_metric': ['auc', 'ams@0'], 'nthread': 4}

# evallist = [(dval, 'eval'), (dtrain, 'train')]

In [21]:
num_round = 500
seed = 42
cv_results = xgb.cv(
    param, 
    dtrain, 
    num_boost_round=num_round,
    seed=seed, 
    nfold=5,
)


In [22]:
cv_results

,train-auc-mean,train-auc-std,train-ams@0-mean,train-ams@0-std,test-auc-mean,test-auc-std,test-ams@0-mean,test-ams@0-std
0,0.906710,0.003411,29.642982,0.535671,0.807638,0.017847,14.620632,0.667569
1,0.949919,0.006827,30.059559,0.288093,0.822766,0.015248,14.880760,0.584999
2,0.974606,0.002818,30.104237,0.273967,0.829554,0.012916,14.896152,0.595225
3,0.985759,0.002476,30.102702,0.290284,0.836290,0.017383,14.904008,0.572549
4,0.990986,0.002123,30.090755,0.272364,0.843178,0.019771,14.888181,0.540660
...,...,...,...,...,...,...,...,...
495,0.999998,0.000001,30.093527,0.297969,0.888975,0.012655,14.881705,0.557965
496,0.999998,0.000001,30.093527,0.297969,0.889015,0.012705,14.878035,0.562348
497,0.999998,0.000001,30.093527,0.297969,0.889022,0.012699,14.878035,0.562348
498,0.999998,0.000001,30.093527,0.297969,0.888990,0.012735,14.878035,0.562348


## Hyperparameter Tuning

In [23]:
from sklearn.model_selection import GridSearchCV

In [33]:
params = { 'num_class': [2], 
        'max_depth' : [2, 3, 4, 5, 6, 7],
        'learning_rate' : [0.01, 0.05, 0.1],
        'n_estimators' : [100, 500, 1000, 2000],
        'objective': ['multi:softprob'],
        'eval_metric': ['roc_auc_ovo'],
        'gamma': [0.1, 0.5, 1, 2, 5],
        'colsample_bytree': [0.5, 0.7, 1]
}

xgbc = xgb.XGBClassifier(seed = 42, )

clf = GridSearchCV(estimator = xgbc,
                param_grid = params,
                scoring='roc_auc_ovo',
                verbose = 1,
                n_jobs = 10
            
)

print('Setup completed, training now')

clf.fit(train_instances_int, train_labels_int)


Setup completed, training now
Fitting 5 folds for each of 2430 candidates, totalling 12150 fits


/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
4050 fits failed out of a total of 12150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/xgboost/core.py", line 532, in inner_f
    return f(**kwargs)
  File "/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/p

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             n_jobs=10,
             param_grid={'c

In [8]:
from joblib import dump, load

In [36]:
dump(clf, 'xgbCVGridSearchC1_2.joblib')

['xgbCVGridSearchC1_2.joblib']

In [37]:
model = clf.best_estimator_

dump(model, 'xgbBest_CVC1_2.joblib') 

['xgbBest_CVC1_2.joblib']

## Analysis 

In [34]:
df1 = pd.DataFrame(clf.cv_results_)

In [35]:
df1[df1['mean_test_score'] == df1['mean_test_score'].max()]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_eval_metric,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,...,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
41,20.123997,0.055880,0.045213,0.009588,0.5,auc,0.1,0.1,3,1000,...,multi:softprob,"{'colsample_bytree': 0.5, 'eval_metric': 'auc'...",0.924216,0.913445,0.906734,0.942905,0.939965,0.925453,0.014222,1
311,19.170863,0.074919,0.033720,0.002689,0.5,ams@0,0.1,0.1,3,1000,...,multi:softprob,"{'colsample_bytree': 0.5, 'eval_metric': 'ams@...",0.924216,0.913445,0.906734,0.942905,0.939965,0.925453,0.014222,1


In [2]:
Gridsearch = load('xgbCVGridSearch.joblib')

In [11]:
model = Gridsearch.best_estimator_

In [31]:
type(model)

xgboost.sklearn.XGBClassifier

In [32]:
print("Best parameters:", clf.best_params_)
print("Highest AUC: ", (clf.best_score_))


Best parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 1000, 'num_class': 2, 'objective': 'multi:softprob'}
Highest AUC:  0.9261966707035846


In [8]:
df1 = pd.DataFrame(Gridesearch.cv_results_)
# df1.to_csv('xgboost_results.csv')

In [10]:
df1.sort_values(['rank_test_score']).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_num_class,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
44,80.091854,8.578351,0.422838,0.032511,0.5,0.1,10,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.928852,0.929915,0.932881,0.929763,0.933452,0.930973,0.001837,1
41,61.540091,6.199461,0.324311,0.029329,0.5,0.1,8,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.927215,0.928086,0.931972,0.927805,0.931833,0.929382,0.002077,2
29,84.471425,9.330232,0.453256,0.045889,0.5,0.05,10,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",0.925553,0.927085,0.930634,0.926786,0.930606,0.928133,0.002095,3
89,92.961855,7.584533,0.471483,0.070459,0.7,0.1,10,1000,3,multi:softprob,"{'colsample_bytree': 0.7, 'learning_rate': 0.1...",0.926387,0.926844,0.930113,0.926399,0.930192,0.927987,0.001776,4
43,45.372425,1.339221,0.232754,0.016557,0.5,0.1,10,500,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.925062,0.926473,0.929667,0.926238,0.930366,0.927561,0.002073,5


In [11]:
df1.to_csv('xgboost_results.csv')

In [13]:
proba = model.predict_proba(test_instances_int)

In [14]:
test = model.predict(test_instances_int)

In [20]:
proba.tofile('../../../Files/models/xgb_proba.txt', sep=' ')

In [21]:
test.tofile('../../../Files/models/xgb_test.txt', sep=' ')

## Test trained model

In [48]:
model = load('xgbBest_CVC2.joblib')

In [50]:
y_test = model.predict_proba(test_instances_int)

In [51]:
y_test.tofile('../../../Files/models/xgboostC2.txt', sep=' ')

In [35]:
y_test.shape

(2743, 2)